# 知识文件扫描与智能分组 - 带详细注释版

## 📚 学习目标

这个notebook会**边做边学**,详细解释:
- 每个Python特性的作用
- 每个API的用法
- 为什么这样写
- 常见问题和替代方案

**适合人群**: Python初学者、LangChain初学者

**学习方式**: 按顺序执行每个cell,仔细阅读注释

---


## 1. 导入依赖 - 每个import的作用

**import的作用**: 导入Python的标准库或第三方库,让我们可以使用它们的功能


In [ ]:
# ============================================================
# re: 正则表达式库 (Regular Expression)
# ============================================================
# 作用: 处理文本,查找、替换、提取特定模式的字符串
# 例如: 从"01第一节"中提取数字"01"
# 官方文档: https://docs.python.org/3/library/re.html
# 口诀: 正则表达式 = 文本的查找工具
import re

# ============================================================
# json: JSON格式处理库
# ============================================================
# 作用: 将Python字典转换为JSON格式(用于保存文件)
# 或将JSON文件读取为Python字典(用于读取文件)
# 官方文档: https://docs.python.org/3/library/json.html
# 口诀: JSON = 数据的交换格式(像Excel一样,但更轻量)
import json

# ============================================================
# Path: 文件路径处理类 (来自pathlib)
# ============================================================
# 作用: 比字符串(str)更安全地处理文件路径
# 为什么用Path而不是str?
#   - str: "/user/file.txt" (容易出错,windows/mac路径不同)
#   - Path: Path("/user/file.txt") (自动适配操作系统)
# 官方文档: https://docs.python.org/3/library/pathlib.html
# 口诀: Path = 智能的文件路径(会自动处理斜杠方向)
from pathlib import Path

# ============================================================
# typing: 类型提示库
# ============================================================
# 作用: 告诉别人(和IDE)这个变量应该是什么类型
# 例如: def add(a: int, b: int) -> int:
#       表示: a和b应该是整数,返回值也是整数
# Dict = 字典类型, List = 列表类型, Tuple = 元组类型等
# 官方文档: https://docs.python.org/3/library/typing.html
# 口诀: Type Hints = 给代码加上"说明书"
from typing import Dict, List, Tuple, Optional, Set

# ============================================================
# dataclass: 数据类装饰器
# ============================================================
# 作用: 自动生成__init__等方法,减少重复代码
# 为什么用dataclass?
#   - 传统方式: 要自己写__init__, __repr__等,很繁琐
#   - dataclass: 自动生成,只需定义属性
# 官方文档: https://docs.python.org/3/library/dataclasses.html
# 口诀: dataclass = 自动写初始化代码的工具
from dataclasses import dataclass, asdict

# ============================================================
# defaultdict: 带默认值的字典
# ============================================================
# 作用: 访问不存在的key时,自动创建默认值(不会报错)
# 普通字典: d = {}; d['a'] → KeyError 错误!
# defaultdict: d = defaultdict(dict); d['a'] → {} 自动创建!
# 官方文档: https://docs.python.org/3/library/collections.html
# 口诀: defaultdict = 永远不会报"key不存在"错误的字典
from collections import defaultdict

# ============================================================
# SequenceMatcher: 序列相似度计算器
# ============================================================
# 作用: 计算两个字符串有多相似(0-1的分数)
# 例如: "中国经济的三驾马车" vs "中国经济三驾马车" → 0.95 (很相似)
#      "三驾马车" vs "消费" → 0.0 (完全不同)
# 算法: 基于最长公共子序列(LCS)
# 官方文档: https://docs.python.org/3/library/difflib.html
# 口诀: SequenceMatcher = 文本相似度打分器
from difflib import SequenceMatcher

# ============================================================
# IntEnum: 整数枚举类
# ============================================================
# 作用: 定义一组有意义的常量(避免使用魔法数字)
# 为什么用枚举?
#   - 不用枚举: priority = 1 (1是什么意思?不清楚!)
#   - 用枚举: priority = FilePriority.PDF_NOTE (一看就懂!)
# 官方文档: https://docs.python.org/3/library/enum.html
# 口诀: Enum = 给数字取名字,让代码更易读
from enum import IntEnum

print("✅ 依赖导入完成")


## 2. 数据结构定义

### 知识点: 为什么要定义数据结构?

**类比**: 数据结构就像"表格的列名"

|  文件路径  |  原始名  |  清洗后名  |  序号  |  优先级  |
|-----------|---------|-----------|-------|---------|
| /path/a.pdf | 01...pdf | 01第一节 |   1   | PDF_NOTE |

如果没有数据结构,代码会很混乱:
```python
# 混乱的方式 (不推荐)
file_data = ["/path/a.pdf", "01...pdf", "01第一节", 1, "PDF_NOTE"]
# 问题: file_data[2]是什么? 要记住顺序,容易出错!
```

有了数据结构:
```python
# 清晰的方式 (推荐)
file_info = FileInfo(path=..., original_name=..., ...)
# 优势: file_info.cleaned_name 一看就懂!
```


In [ ]:
# ============================================================
# 数据结构1: FilePriority - 文件优先级枚举
# ============================================================

class FilePriority(IntEnum):
    """
    文件优先级枚举

    为什么要定义优先级?
    答: 同一知识块有3个文件(pdf/doc/ppt),我们只想处理最完整的那个

    优先级规则(数字越小优先级越高):
    1. PDF笔记 → 课程笔记,内容最完整
    2. Word文档 → 详细文字说明
    3. 普通PDF → 可能是扫描件
    4. PPT → 图表多但文字少

    IntEnum的作用:
    - 继承自int,可以当数字用 (FilePriority.PDF_NOTE == 1 → True)
    - 有名字,可读性好 (priority.name → "PDF_NOTE")
    """
    PDF_NOTE = 1      # PDF笔记：内容最完整
    WORD_DOC = 2      # Word文档：详细说明
    PDF_REGULAR = 3   # 普通PDF
    POWERPOINT = 4    # PPT：图表多但文字少
    UNKNOWN = 99      # 未知格式

# 测试一下
print(f"PDF笔记的优先级数字: {FilePriority.PDF_NOTE}")  # 输出: 1
print(f"PDF笔记的优先级名称: {FilePriority.PDF_NOTE.name}")  # 输出: PDF_NOTE
print(f"PDF笔记 < Word? {FilePriority.PDF_NOTE < FilePriority.WORD_DOC}")  # True


In [ ]:
# ============================================================
# 数据结构2: FileInfo - 文件信息类
# ============================================================

@dataclass  # 装饰器: 告诉Python"这是一个数据类"
class FileInfo:
    """
    文件信息数据类

    dataclass的魔法:
    ----------------
    没有@dataclass时,你要写:
        def __init__(self, path, original_name, ...):
            self.path = path
            self.original_name = original_name
            ...
        def __repr__(self):
            return f"FileInfo(...)"

    有了@dataclass,Python自动帮你生成上面的代码!

    Type Hints解释:
    ---------------
    - Path: pathlib.Path类型(文件路径)
    - str: 字符串类型
    - int: 整数类型
    - FilePriority: 我们刚定义的枚举类型

    这些类型提示的作用:
    - 告诉IDE这是什么类型(会有自动补全)
    - 告诉读代码的人这个变量应该是什么
    - 可以用mypy等工具检查类型错误
    """

    # 文件路径 (Path类型,不是str!)
    path: Path

    # 原始文件名 (例如: "01第一节[防断更]_笔记.pdf")
    original_name: str

    # 清洗后的文件名 (例如: "01第一节")
    cleaned_name: str

    # 序号 (例如: 1, 2, 3...)
    sequence: int

    # 序号字符串 (例如: "01", "02", "03")
    # 为什么要保留字符串? 因为"01"和"1"不同,"01"保留了前导零
    sequence_str: str

    # 文件优先级 (例如: FilePriority.PDF_NOTE)
    priority: FilePriority

    def __repr__(self) -> str:
        """
        自定义打印格式

        为什么要重写__repr__?
        答: 默认的打印格式太长,不方便查看

        __repr__的作用:
        当你print(file_info)时,会调用这个方法

        返回值类型 -> str 表示: 这个方法返回字符串
        """
        return f"FileInfo({self.sequence_str}_{self.cleaned_name[:20]}..., priority={self.priority.name})"

# 测试一下FileInfo
test_file = FileInfo(
    path=Path("/test/file.pdf"),
    original_name="01第一节.pdf",
    cleaned_name="01第一节",
    sequence=1,
    sequence_str="01",
    priority=FilePriority.PDF_NOTE
)

print("测试FileInfo:")
print(test_file)  # 会调用__repr__方法
print(f"访问属性: {test_file.cleaned_name}")  # 访问cleaned_name属性


In [ ]:
# ============================================================
# 数据结构3: KnowledgeGroup - 知识块分组类
# ============================================================

@dataclass
class KnowledgeGroup:
    """
    知识块分组数据类

    一个知识块 = 同一个主题的多个文件
    例如: "01第一节 三驾马车" 这个知识块包含:
        - 01第一节_笔记.pdf
        - 01第一节.doc
        - 01第一节.pptx

    Type Hints高级用法:
    -------------------
    - List[FileInfo]: 表示"FileInfo对象的列表"
    - List[str]: 表示"字符串的列表"
    - Dict: 表示"字典"
    """

    # 组的唯一标识 (例如: "01_中国经济的三驾马车")
    group_key: str

    # 知识主题 (例如: "中国经济的三驾马车")
    topic: str

    # 序号 (例如: 1)
    sequence: int

    # 该组的所有文件 (List[FileInfo]表示:这是一个列表,里面装的是FileInfo对象)
    files: List[FileInfo]

    # 主文件 (优先处理的文件,通常是PDF笔记)
    primary_file: FileInfo

    # 文件类型列表 (例如: ['.pdf', '.doc', '.pptx'])
    file_types: List[str]

    def to_dict(self) -> Dict:
        """
        转换为字典(用于JSON序列化)

        为什么需要to_dict?
        答: dataclass对象不能直接保存为JSON,需要先转换为字典

        返回值 -> Dict 表示: 返回字典类型

        str(path)的作用:
        - Path对象不能直接序列化为JSON
        - str(Path("/a/b.pdf")) → "/a/b.pdf" (转换为字符串)
        """
        return {
            "group_key": self.group_key,
            "topic": self.topic,
            "sequence": self.sequence,
            "primary_file": str(self.primary_file.path),  # Path转字符串
            "files": [str(f.path) for f in self.files],   # 列表推导式
            "file_types": self.file_types,
        }

# 测试一下KnowledgeGroup
test_group = KnowledgeGroup(
    group_key="01_测试",
    topic="测试主题",
    sequence=1,
    files=[test_file],  # 使用之前创建的test_file
    primary_file=test_file,
    file_types=['.pdf']
)

print("\n测试KnowledgeGroup:")
print(f"主题: {test_group.topic}")
print(f"文件数: {len(test_group.files)}")
print(f"转换为字典:\n{json.dumps(test_group.to_dict(), indent=2, ensure_ascii=False)}")

print("\n✅ 数据结构定义完成")


## 3. KnowledgeOrganizer 核心类 - 方法详解

### 类的设计理念

**单一职责原则 (SRP)**:
- 这个类只做一件事: 组织知识文件
- 不负责加载文件内容(那是DocumentLoader的工作)
- 不负责提取知识(那是KnowledgeExtractor的工作)

**类 = 对象的模板**:
- 类定义了对象有哪些属性和方法
- 通过类创建对象: `organizer = KnowledgeOrganizer(...)`


In [ ]:
# ============================================================
# KnowledgeOrganizer 类的开头部分
# ============================================================

class KnowledgeOrganizer:
    """
    知识文件智能组织器

    这个类的工作流程:
    1. 扫描目录 → 找到所有pdf/doc/pptx文件
    2. 清洗文件名 → 去除[防断更]等噪音
    3. 提取序号 → 识别01, 02等
    4. 计算相似度 → 判断哪些文件属于同一知识块
    5. 智能分组 → 把相似的文件归为一组
    6. 选择主文件 → 每组选优先级最高的文件
    """

    # ============================================================
    # 类属性 (Class Attributes)
    # ============================================================
    # 类属性 vs 实例属性:
    # - 类属性: 所有对象共享,定义在__init__外面
    # - 实例属性: 每个对象独有,定义在__init__里面
    #
    # 为什么这里用类属性?
    # 答: 支持的文件扩展名和噪音模式对所有对象都一样,没必要每个对象都复制一份

    # Set[str] 表示: 这是一个集合,里面装的是字符串
    # Set vs List的区别:
    # - Set: 无序,不能重复,查找快 → 适合"判断某个元素是否存在"
    # - List: 有序,可以重复,查找慢 → 适合"需要顺序"的场景
    SUPPORTED_EXTENSIONS: Set[str] = {'.pdf', '.doc', '.docx', '.ppt', '.pptx'}

    # List[str] 表示: 这是一个列表,里面装的是字符串
    # 这些字符串是正则表达式模式(pattern)
    # 用于匹配并删除文件名中的噪音
    NOISE_PATTERNS: List[str] = [
        r'\[防断更.*?\]',      # 匹配 [防断更微coc36666]
        r'\[.*?微.*?\]',        # 匹配 任何包含"微"的方括号
        r'_\d{14}',             # 匹配 _20250706193405 (14位数字)
        r'_\d{8}',              # 匹配 _20250706 (8位数字)
        r'_笔记',                # 匹配 _笔记
        r'\s*\(.*?\)\s*',       # 匹配 (任何括号内容)
    ]

print("✅ 类定义开头部分完成")


### __init__方法 - 初始化

**init的作用**: 创建对象时自动调用,用于初始化对象的属性

```python
organizer = KnowledgeOrganizer("/path/to/dir")
# 上面这行代码会自动调用__init__方法
```


In [ ]:
# __init__方法的完整示例(独立展示,便于理解)

def __init__(
    self,                          # self = 对象本身(Python自动传入,不需要手动传)
    knowledge_base_dir: str | Path,  # 知识库目录 (str或Path都可以)
    similarity_threshold: float = 0.7,  # 相似度阈值,默认0.7
    verbose: bool = True            # 是否打印日志,默认True
):
    """
    初始化方法

    参数解释:
    -----------
    self:
        - 对象本身
        - Python自动传入,调用时不需要写
        - 通过self访问对象的属性: self.xxx

    knowledge_base_dir: str | Path
        - str | Path 表示: 可以传字符串或Path对象
        - 例如: "/path/to/dir" 或 Path("/path/to/dir")
        - Python 3.10+ 的新语法,也可写成 Union[str, Path]

    similarity_threshold: float = 0.7
        - = 0.7 表示默认值
        - 如果不传这个参数,就用0.7
        - 调用时可以改: KnowledgeOrganizer(..., similarity_threshold=0.8)

    verbose: bool = True
        - bool = 布尔类型 (True或False)
        - 控制是否打印日志
    """

    # ============================================================
    # 步骤1: 保存参数到对象属性
    # ============================================================
    # self.xxx = yyy 表示: 把yyy保存到对象的xxx属性中
    # 这样其他方法就可以通过 self.xxx 访问到这个值

    self.knowledge_base_dir = Path(knowledge_base_dir)  # 统一转换为Path对象
    self.similarity_threshold = similarity_threshold
    self.verbose = verbose

    # ============================================================
    # 步骤2: 验证目录是否有效
    # ============================================================
    # 防御性编程: 检查输入是否合法,如果不合法就报错
    # 这样可以在问题发生时立即发现,而不是等到后面出错

    if not self.knowledge_base_dir.exists():
        # .exists()方法: 检查路径是否存在
        # not表示取反: 如果不存在,就执行下面的代码
        raise ValueError(f"目录不存在: {self.knowledge_base_dir}")
        # raise: 抛出异常(报错)
        # ValueError: 值错误(参数不合法)
        # f-string: f"..." 可以在字符串中插入变量

    if not self.knowledge_base_dir.is_dir():
        # .is_dir()方法: 检查路径是否是目录(而不是文件)
        raise ValueError(f"不是目录: {self.knowledge_base_dir}")

    # ============================================================
    # 步骤3: 打印初始化成功信息
    # ============================================================
    self._log(f"✅ 初始化完成: {self.knowledge_base_dir}")
    # _log是自定义的辅助方法(下面会定义)
    # 方法名前有下划线_表示: 这是"私有方法",仅供内部使用

print("✅ __init__方法解释完成")

# ============================================================
# 知识点补充: str | Path vs Union[str, Path]
# ============================================================
print("\n💡 知识点: 类型提示的两种写法")
print("Python 3.10+: str | Path")
print("Python 3.9-:  Union[str, Path]")
print("两种写法功能完全相同,新语法更简洁")


### 核心方法实现


In [ ]:
# 完整的KnowledgeOrganizer类实现(适度详细注释版)

class KnowledgeOrganizer:
    """知识文件智能组织器"""

    SUPPORTED_EXTENSIONS: Set[str] = {'.pdf', '.doc', '.docx', '.ppt', '.pptx'}
    NOISE_PATTERNS: List[str] = [
        r'\[防断更.*?\]', r'\[.*?微.*?\]', r'_\d{14}',
        r'_\d{8}', r'_笔记', r'\s*\(.*?\)\s*',
    ]

    def __init__(self, knowledge_base_dir: str | Path,
                 similarity_threshold: float = 0.7, verbose: bool = True):
        self.knowledge_base_dir = Path(knowledge_base_dir)
        self.similarity_threshold = similarity_threshold
        self.verbose = verbose

        if not self.knowledge_base_dir.exists():
            raise ValueError(f"目录不存在: {self.knowledge_base_dir}")
        if not self.knowledge_base_dir.is_dir():
            raise ValueError(f"不是目录: {self.knowledge_base_dir}")

        self._log(f"✅ 初始化完成: {self.knowledge_base_dir}")

    def _log(self, message: str) -> None:
        """内部日志方法"""
        if self.verbose:
            print(message)

    def clean_filename(self, filename: str) -> str:
        """
        清洗文件名,去除噪音

        参数: filename - 原始文件名(含扩展名)
        返回: 清洗后的文件名(不含扩展名)

        处理流程:
        1. Path(filename).stem → 去除扩展名
        2. re.sub(pattern, '', name) → 应用所有噪音模式
        3. re.sub(r'\\s+', ' ', name).strip() → 清理空格
        """
        name = Path(filename).stem
        for pattern in self.NOISE_PATTERNS:
            name = re.sub(pattern, '', name)
        name = re.sub(r'\\s+', ' ', name).strip()
        return name

    def extract_sequence_number(self, filename: str) -> Tuple[int, str]:
        """
        提取文件名开头的序号

        参数: filename - 文件名
        返回: (序号整数, 序号字符串) 或 (999999, "") 如果没有序号

        示例: "01第一节" → (1, "01")
        """
        match = re.match(r'^(\\d+)', filename)
        if match:
            seq_str = match.group(1)
            return (int(seq_str), seq_str)
        return (999999, "")  # 没有序号的文件排到最后

    def calculate_similarity(self, str1: str, str2: str) -> float:
        """
        计算字符串相似度

        参数: str1, str2 - 两个字符串
        返回: 相似度分数 (0-1)

        算法: difflib.SequenceMatcher基于LCS(最长公共子序列)
        """
        return SequenceMatcher(None, str1, str2).ratio()

    def get_file_priority(self, file_path: Path) -> FilePriority:
        """
        获取文件优先级

        规则:
        - PDF笔记(文件名含"笔记") → 最高优先级
        - Word文档 → 次之
        - 普通PDF → 再次
        - PPT → 最低
        """
        name_lower = file_path.name.lower()
        suffix = file_path.suffix.lower()

        if '笔记' in name_lower and suffix == '.pdf':
            return FilePriority.PDF_NOTE
        if suffix in ['.doc', '.docx']:
            return FilePriority.WORD_DOC
        if suffix == '.pdf':
            return FilePriority.PDF_REGULAR
        if suffix in ['.ppt', '.pptx']:
            return FilePriority.POWERPOINT
        return FilePriority.UNKNOWN

    def create_file_info(self, file_path: Path) -> FileInfo:
        """创建FileInfo对象(汇总所有文件信息)"""
        original_name = file_path.name
        cleaned_name = self.clean_filename(original_name)
        sequence, sequence_str = self.extract_sequence_number(cleaned_name)
        priority = self.get_file_priority(file_path)

        return FileInfo(
            path=file_path, original_name=original_name,
            cleaned_name=cleaned_name, sequence=sequence,
            sequence_str=sequence_str, priority=priority
        )

    def group_files_by_similarity(self, files: List[FileInfo]) -> Dict[str, KnowledgeGroup]:
        """
        核心算法: 根据相似度分组文件

        流程:
        1. 遍历每个文件
        2. 查找相似度>=threshold且序号相同的文件
        3. 归为一组
        4. 选择优先级最高的作为主文件

        时间复杂度: O(n²)
        空间复杂度: O(n)
        """
        if not files:
            return {}

        groups: Dict[str, KnowledgeGroup] = {}
        processed: Set[Path] = set()

        for i, file1 in enumerate(files):
            if file1.path in processed:
                continue

            group_key = f"{file1.sequence_str}_{file1.cleaned_name[:20]}"
            group_files = [file1]
            processed.add(file1.path)

            # 查找相似文件
            for file2 in files[i+1:]:
                if file2.path in processed:
                    continue

                if file1.sequence == file2.sequence:
                    similarity = self.calculate_similarity(
                        file1.cleaned_name, file2.cleaned_name
                    )

                    if similarity >= self.similarity_threshold:
                        group_files.append(file2)
                        processed.add(file2.path)
                        self._log(f"  ↳ 相似度={similarity:.2f}: {file2.original_name}")

            # 按优先级排序,选主文件
            group_files.sort(key=lambda f: (f.priority.value, f.original_name))
            primary_file = group_files[0]

            group = KnowledgeGroup(
                group_key=group_key, topic=file1.cleaned_name,
                sequence=file1.sequence, files=group_files,
                primary_file=primary_file,
                file_types=[f.path.suffix for f in group_files]
            )

            groups[group_key] = group
            self._log(f"✓ 知识块: {file1.cleaned_name} ({len(group_files)}个文件)")

        return groups

    def scan_and_organize(self) -> Dict[str, Dict[str, KnowledgeGroup]]:
        """
        主方法: 扫描并组织目录

        返回: {domain_name: {group_key: KnowledgeGroup}}
        """
        result: Dict[str, Dict[str, KnowledgeGroup]] = defaultdict(dict)

        self._log(f"\\n{'='*80}\\n📚 开始扫描: {self.knowledge_base_dir}\\n{'='*80}\\n")

        # 获取所有文件
        all_files: List[Path] = []
        for ext in self.SUPPORTED_EXTENSIONS:
            all_files.extend(self.knowledge_base_dir.glob(f"*{ext}"))

        if not all_files:
            self._log("⚠️  未找到支持的文件")
            return dict(result)

        self._log(f"📂 找到 {len(all_files)} 个文件\\n")

        # 创建FileInfo → 分组 → 排序
        file_infos = [self.create_file_info(f) for f in all_files]
        groups = self.group_files_by_similarity(file_infos)
        sorted_groups = dict(sorted(groups.items(), key=lambda x: x[1].sequence))

        domain_name = self.knowledge_base_dir.name
        result[domain_name] = sorted_groups

        self._log(f"\\n{'='*80}\\n✅ 完成: {len(sorted_groups)} 个知识块\\n{'='*80}\\n")

        return dict(result)

    def print_organization(self) -> Dict[str, Dict[str, KnowledgeGroup]]:
        """扫描并打印组织结构(用于检查)"""
        organized = self.scan_and_organize()

        print("\\n" + "="*80)
        print("📊 知识库组织结构")
        print("="*80 + "\\n")

        for domain, groups in organized.items():
            print(f"🎯 领域: {domain}")
            print("-" * 80)

            for group_key, group in groups.items():
                print(f"\\n  📖 #{group.sequence}: {group.topic}")
                print(f"     主文件: {group.primary_file.original_name}")
                print(f"     优先级: {group.primary_file.priority.name}")
                print(f"     文件数: {len(group.files)}")

                if len(group.files) > 1:
                    print(f"     所有文件:")
                    for f in group.files:
                        print(f"       • {f.original_name} [{f.priority.name}]")

        print("\\n" + "="*80 + "\\n")
        return organized

    def export_to_json(self, output_path: str | Path) -> None:
        """导出为JSON"""
        organized = self.scan_and_organize()

        export_data = {}
        for domain, groups in organized.items():
            export_data[domain] = {
                group_key: group.to_dict()
                for group_key, group in groups.items()
            }

        output_path = Path(output_path)
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(export_data, f, ensure_ascii=False, indent=2)

        self._log(f"✅ 已导出到: {output_path}")


print("✅ KnowledgeOrganizer 类完整定义完成")


## 4. 使用示例


In [ ]:
# 初始化组织器
organizer = KnowledgeOrganizer(
    knowledge_base_dir="/Users/zhou/Project/AnalystChain/jupyter_notebook/macroeconomic_analysis/knowledge_base",
    similarity_threshold=0.7,
    verbose=True
)


In [ ]:
# 打印组织结构
organized = organizer.print_organization()


## 5. 总结

### ✅ 核心知识点

1. **Type Hints**: 给代码加"说明书"
2. **dataclass**: 自动生成初始化代码
3. **Path**: 智能文件路径处理
4. **SequenceMatcher**: 字符串相似度计算
5. **正则表达式**: 文本模式匹配

### 📚 下一步学习

- **步骤2**: 文档加载与清洗 (DocumentLoader)
- **步骤3**: LLM提取结构化知识 (KnowledgeExtractor)
- **步骤4**: 向量化存储 (VectorStoreManager)

### 💡 学习建议

1. 运行每个cell,看看输出
2. 修改参数,观察变化
3. 遇到不懂的API,查官方文档或问我
4. 抄到自己的notebook,边抄边理解
